In [1]:
#ALL IMPORTS
import numpy as np
import pandas as pd

In [2]:
# READING DATA
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [18]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.name=name
        self.teams = {i for i in self.data.HomeTeam}
    
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams:
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams:
            loseTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
        
        return None
    
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams:
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamsTotalGoals(self):
        #TODO
        teamTotalGoals = dict()
        for i in self.teams:
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTHG
            
        return None
    
    def Test(self):
        print("Passed")

In [19]:
season1718 = Football(season17_18,"Hooaeg 17-18")
print(season1718.getTeamLoses())

{'Liverpool': 5, 'Man United': 7, 'Everton': 15, 'Watford': 19, 'West Brom': 19, 'Crystal Palace': 16, 'Tottenham': 7, 'Huddersfield': 19, 'Man City': 2, 'Leicester': 15, 'Brighton': 16, 'Bournemouth': 16, 'Swansea': 21, 'Newcastle': 18, 'Arsenal': 13, 'West Ham': 16, 'Southampton': 16, 'Burnley': 12, 'Stoke': 19, 'Chelsea': 10}
